# FedPSO (clientes com dados completos)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset, random_split
import copy
import torchvision.models as models
import random
import csv

#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# Configurações
num_clientes = 10 
num_rodadas = 30   
epocas_cliente = 5  #para modelos pré-treinados só uma única época do cliente
BATCH_SIZE = 64 
inertia = 0.9  
c1 = 0.8  
c2 = 0.9


# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def init_model(architecture="squeezenet"):
    if architecture == "squeezenet":
        #model = models.squeezenet1_1(weights=None)
        model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
    elif architecture == "mobilenet_v2":
        #model = models.mobilenet_v2(weights=None)
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    elif architecture == "resnet18":
        #model = models.resnet18(weights=None)
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif architecture == "alexnet":
        #model = models.alexnet(weights=None)
        model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
    else:
        raise ValueError("Unsupported architecture. Choose from 'squeezenet', 'mobilenet_v2', 'resnet18', 'alexnet'.")
    return model.to(device)

# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

# Função para salvar os resultados em um arquivo CSV
def write_csv(algorithm_name, architecture, results):
    file_name = f'{algorithm_name}_{architecture}_preTrei_CIFAR10_output.csv'
    with open(file_name, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Epoch", "Test Loss", "Test Accuracy"])
        for i, result in enumerate(results):
            writer.writerow([i + 1] + result)

class Particle:
    def __init__(self, particle_id, model, data, architecture="squeezenet"):
        self.particle_id = particle_id
        self.model = model.to(device)
        self.data = data  # `self.data` agora armazena o dataset, não o DataLoader
        self.local_best_model = copy.deepcopy(model.state_dict())
        self.local_best_score = float('inf')
        self.velocity = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-5)

    def train_particle(self, criterion):
        self.model.train()
        train_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=True)
        for _ in range(epocas_cliente):
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

        val_loss = self.evaluate_loss(criterion)
        if val_loss < self.local_best_score:
            self.local_best_score = val_loss
            self.local_best_model = copy.deepcopy(self.model.state_dict())
        return self.particle_id, val_loss

    def evaluate_loss(self, criterion):
        self.model.eval()
        total_loss = 0
        val_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=False)
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(val_loader)

    def pso_update(self, global_best_model, inertia, c1, c2):
        for name, param in self.model.named_parameters():
            if param.grad is None:
                continue
            local_rand = random.random()
            global_rand = random.random()
            self.velocity[name] = (
                inertia * self.velocity[name]
                + c1 * local_rand * (self.local_best_model[name].to(device) - param.data)
                + c2 * global_rand * (global_best_model[name].to(device) - param.data)
            )
            param.data += self.velocity[name]

def train_federated(architecture="squeezenet"):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloaders = [create_subset(trainset, 5000) for _ in range(num_clientes)]

    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    server_model = init_model(architecture)
    particles = [Particle(i, copy.deepcopy(server_model), trainloaders[i], architecture) for i in range(num_clientes)]
    criterion = nn.CrossEntropyLoss()

    global_best_score = float('inf')
    global_best_model = copy.deepcopy(server_model.state_dict())
    server_evaluate_acc = []

    for epoch in range(num_rodadas):
        server_result = []
        for particle in particles:
            if epoch > 0:
                particle.pso_update(global_best_model, inertia, c1, c2)
            pid, val_loss = particle.train_particle(criterion)
            server_result.append((pid, val_loss))

        best_particle = min(server_result, key=lambda x: x[1])
        global_best_score = best_particle[1]
        global_best_model = copy.deepcopy(particles[best_particle[0]].local_best_model)

        print(f'Epoch {epoch + 1}/{num_rodadas}, Best Particle Loss: {global_best_score:.4f}')

        server_model.load_state_dict(global_best_model)
        server_model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = server_model(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
        test_loss = total_loss / len(testloader)
        test_accuracy = correct / len(testset)
        server_evaluate_acc.append([test_loss, test_accuracy])
        print(f'Epoch {epoch + 1}/{num_rodadas}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    write_csv("FedPSO", architecture, server_evaluate_acc)

# Exemplo de execução
#train_federated("squeezenet")

## SqueezeNet

In [3]:
train_federated("squeezenet")  

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 1.1665
Epoch 1/30, Test Loss: 1.2113, Test Accuracy: 0.5811
Epoch 2/30, Best Particle Loss: 0.7844
Epoch 2/30, Test Loss: 0.8843, Test Accuracy: 0.6911
Epoch 3/30, Best Particle Loss: 0.6453
Epoch 3/30, Test Loss: 0.7751, Test Accuracy: 0.7297
Epoch 4/30, Best Particle Loss: 0.5623
Epoch 4/30, Test Loss: 0.7349, Test Accuracy: 0.7561
Epoch 5/30, Best Particle Loss: 0.5300
Epoch 5/30, Test Loss: 0.7155, Test Accuracy: 0.7477
Epoch 6/30, Best Particle Loss: 0.4858
Epoch 6/30, Test Loss: 0.6917, Test Accuracy: 0.7625
Epoch 7/30, Best Particle Loss: 0.4101
Epoch 7/30, Test Loss: 0.6436, Test Accuracy: 0.7753
Epoch 8/30, Best Particle Loss: 0.3890
Epoch 8/30, Test Loss: 0.6371, Test Accuracy: 0.7798
Epoch 9/30, Best Particle Loss: 0.3888
Epoch 9/30, Test Loss: 0.6698, Test Accuracy: 0.7746
Epoch 10/30, Best Particle Loss: 0.2924
Epoch 10/30, Test Loss: 0.6001, Test Accuracy: 0.8004
Ep

## MobileNet_v2

In [4]:
train_federated("mobilenet_v2") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.4158
Epoch 1/30, Test Loss: 0.5587, Test Accuracy: 0.8188
Epoch 2/30, Best Particle Loss: 0.1604
Epoch 2/30, Test Loss: 0.4147, Test Accuracy: 0.8608
Epoch 3/30, Best Particle Loss: 0.0591
Epoch 3/30, Test Loss: 0.3742, Test Accuracy: 0.8713
Epoch 4/30, Best Particle Loss: 0.0251
Epoch 4/30, Test Loss: 0.3764, Test Accuracy: 0.8753
Epoch 5/30, Best Particle Loss: 0.0113
Epoch 5/30, Test Loss: 0.3845, Test Accuracy: 0.8740
Epoch 6/30, Best Particle Loss: 0.0070
Epoch 6/30, Test Loss: 0.3754, Test Accuracy: 0.8774
Epoch 7/30, Best Particle Loss: 0.0041
Epoch 7/30, Test Loss: 0.3921, Test Accuracy: 0.8774
Epoch 8/30, Best Particle Loss: 0.0027
Epoch 8/30, Test Loss: 0.3911, Test Accuracy: 0.8773
Epoch 9/30, Best Particle Loss: 0.0045
Epoch 9/30, Test Loss: 0.3911, Test Accuracy: 0.8773
Epoch 10/30, Best Particle Loss: 0.0022
Epoch 10/30, Test Loss: 0.4181, Test Accuracy: 0.8757
Ep

## Resnet18

In [5]:
train_federated("resnet18") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.2083
Epoch 1/30, Test Loss: 0.4253, Test Accuracy: 0.8616
Epoch 2/30, Best Particle Loss: 0.0464
Epoch 2/30, Test Loss: 0.3540, Test Accuracy: 0.8820
Epoch 3/30, Best Particle Loss: 0.0132
Epoch 3/30, Test Loss: 0.3285, Test Accuracy: 0.8918
Epoch 4/30, Best Particle Loss: 0.0061
Epoch 4/30, Test Loss: 0.3276, Test Accuracy: 0.8918
Epoch 5/30, Best Particle Loss: 0.0048
Epoch 5/30, Test Loss: 0.3503, Test Accuracy: 0.8890
Epoch 6/30, Best Particle Loss: 0.0053
Epoch 6/30, Test Loss: 0.3503, Test Accuracy: 0.8890
Epoch 7/30, Best Particle Loss: 0.0032
Epoch 7/30, Test Loss: 0.3471, Test Accuracy: 0.8909
Epoch 8/30, Best Particle Loss: 0.0201
Epoch 8/30, Test Loss: 0.3471, Test Accuracy: 0.8909
Epoch 9/30, Best Particle Loss: 0.0046
Epoch 9/30, Test Loss: 0.3471, Test Accuracy: 0.8909
Epoch 10/30, Best Particle Loss: 0.0212
Epoch 10/30, Test Loss: 0.4005, Test Accuracy: 0.8797
Ep

## AlexNet

In [6]:
train_federated("alexnet") 

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Best Particle Loss: 0.5172
Epoch 1/30, Test Loss: 0.6761, Test Accuracy: 0.7662
Epoch 2/30, Best Particle Loss: 0.3087
Epoch 2/30, Test Loss: 0.5683, Test Accuracy: 0.8014
Epoch 3/30, Best Particle Loss: 0.2269
Epoch 3/30, Test Loss: 0.4867, Test Accuracy: 0.8289
Epoch 4/30, Best Particle Loss: 0.1820
Epoch 4/30, Test Loss: 0.4742, Test Accuracy: 0.8393
Epoch 5/30, Best Particle Loss: 0.1245
Epoch 5/30, Test Loss: 0.5110, Test Accuracy: 0.8272
Epoch 6/30, Best Particle Loss: 0.0939
Epoch 6/30, Test Loss: 0.4958, Test Accuracy: 0.8364
Epoch 7/30, Best Particle Loss: 0.0715
Epoch 7/30, Test Loss: 0.5572, Test Accuracy: 0.8317
Epoch 8/30, Best Particle Loss: 0.0365
Epoch 8/30, Test Loss: 0.5250, Test Accuracy: 0.8404
Epoch 9/30, Best Particle Loss: 0.0239
Epoch 9/30, Test Loss: 0.5429, Test Accuracy: 0.8454
Epoch 10/30, Best Particle Loss: 0.0180
Epoch 10/30, Test Loss: 0.5597, Test Accuracy: 0.8399
Ep